# Sintetic Software Used  To Analysis of Cassandra's Performance

## Imports

In [36]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import time

## Database configuration

cluster = Cluster(['localhost'], auth_provider=PlainTextAuthProvider('cassandra', 'cassandra'))
session = cluster.connect()

# Create KeySpace
session.execute('CREATE KEYSPACE IF NOT EXISTS analysis WITH replication = {\'class\': \'SimpleStrategy\', \'replication_factor\': 1}')

# Use KeySpace created
session.set_keyspace('analysis')

# Create Table
session.execute("CREATE TABLE IF NOT EXISTS \"table\" ("+
                "A0 uuid, "+
                "A1 int PRIMARY KEY, "+
                "A2 bigint, "+
                "A3 double, "+
                "A4 varchar, "+
                "A5 text, "+
                "A6 date, "+
                "A7 timestamp )")
rows = session.execute('select * from system_schema.columns')
print('+----------------------------------+')
print('|              table               |')
print('+----------------------------------+')
for row in rows:
    if row.table_name == 'table':
        print('|', row.column_name, '|', row.type, ' '*(9-len(row.type)), '|',  row.kind, ' '*(13-len(row.kind)), '|')
print('+----------------------------------+')

## Meta data

# * R is the number of iterations <br />
# * BI is the size of a block of insertions<br />
# * BU is the size of a block of updates<br />
# * BS is the size of a block of selections<br />

R = 60
BI = BU = BS = 500000
categories = [i+1 for i in range(R)]

## Definition of functions

def insert(control):
    initialTime = time.time()
    for i in range(BI):
        session.execute('INSERT INTO "table" (A0, A1, A2, A3, A4, A5, A6, A7) VALUES ('+
                        'uuid(), '+
                        str(control)+', '+
                        '9223372036854775807, '+
                        '1.2, '+
                        '\'abcdefghijklmnopqrstuvwxy\', '+
                        '\'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer ultricies lorem metus, vel finibus risus convallis sit amet. Nam auctor ex et ipsum euismod, vel consectetur eros blandit. Sed sit amet enim vitae nisi varius molestie. Maecenas in tortor sem. Ut sit amet lobortis erat, ac egestas libero. Nunc id purus sodales, dictum massa gravida, condimentum diam. Maecenas eu vulputate nunc, vitae tempor odio. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Duis vitae turpis quam. Maecenas at dui at justo vehicula scelerisque eu in eros. Sed volutpat, magna nec pretium tincidunt, risus nibh posuere mauris, quis feugiat augue lorem ut nunc. Lorem ipsum dolor sit amet, consectetur adipiscing elit. In eu tellus nec nulla ultricies efficitur. Interdum et malesuada fames ac ante ipsum primis in faucibus. \', '+
                        'toDate(dateof(now())), '+
                        'dateof(now())'+
                        ')')
        control += 1
    return (time.time() - initialTime), control

def update(control):
    initialTime = time.time()
    for i in range(BI):
        session.execute('UPDATE "table" SET '+
                        'A2 = 9223372036854775800, '+
                        'A3 = 1.3, '+
                        'A4 = \'abbdefghijklmnopqrstuvwxy\', '+
                        'A5 = \'Lorem ipsum dolor site amet, consectetur adipiscing elit. Integer ultricies lorem metus, vel finibus risus convallis sit amet. Nam auctor ex et ipsum euismod, vel consectetur eros blandit. Sed sit amet enim vitae nisi varius molestie. Maecenas in tortor sem. Ut sit amet lobortis erat, ac egestas libero. Nunc id purus sodales, dictum massa gravida, condimentum diam. Maecenas eu vulputate nunc, vitae tempor odio. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Duis vitae turpis quam. Maecenas at dui at justo vehicula scelerisque eu in eros. Sed volutpat, magna nec pretium tincidunt, risus nibh posuere mauris, quis feugiat augue lorem ut nunc. Lorem ipsum dolor sit amet, consectetur adipiscing elit. In eu tellus nec nulla ultricies efficitur. Interdum et malesuada fames ac ante ipsum primis in faucibus. \', '+
                        'A6 = toDate(dateof(now())), '+
                        'A7 = dateof(now())'+
                        'WHERE A1 = '+str(control))
        control += 1
    return (time.time() - initialTime), control

def select(control):
    initialTime = time.time()
    for i in range(BI):
        session.execute('SELECT * FROM "table" WHERE A1 = '+str(control))
        control += 1
    return (time.time() - initialTime), control

## Storage

insertTimes = []
updateTimes = []
selectTimes = []

## DO IT

control = 1
for j in range(R):
    control = BI*j + 1
    t, control = insert(control)
    print('Insert', control)
    insertTimes.append(t)
    control = BI*j + 1
    t, control = update(control)
    print('Update', control)
    updateTimes.append(t)
    control = BI*j + 1
    t, control = select(control)
    print('Select', control)
    selectTimes.append(t)

0.5582405004552239